In [1]:
Data = ''''The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.0 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. 1 Introduction Recurrent neural networks, long short-term memory [12] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [29, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [31, 21, 13]. Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head attention and the parameter-free position representation and became the other person involved in nearly every detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating our research. Work performed while at Google Brain. Work performed while at Google Research. 31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA. Recurrent models typically factor computation along the symbol positions of the input and output sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of the previous hidden state ht 1 and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples. Recent work has achieved significant improvements in computational efficiency through factorization tricks [18] and conditional computation [26], while also improving model performance in case of the latter. The fundamental constraint of sequential computation, however, remains. Attention mechanisms have become an integral part of compelling sequence modeling and transduc tion models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [2, 16]. In all but a few cases [22], however, such attention mechanisms are used in conjunction with a recurrent network. In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs. 2 Background The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [20], ByteNet [15] and ConvS2S [8], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes it more difficult to learn dependencies between distant positions [11]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 22, 23, 19]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [28]. To the best of our knowledge, however, the Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [14, 15] and [8]. 3 ModelArchitecture Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 29]. Here, the encoder maps an input sequence of symbol representations (x1 of continuous representations z = (z1 xn) to a sequence zn). Given z, the decoder then generates an output sequence (y1 ym) of symbols one element at a time. At each step the model is auto-regressive [9], consuming the previously generated symbols as additional input when generating the next. The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively. 3.1 Encoder and Decoder Stacks Encoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position 2 Figure 1: The Transformer- model architecture. wise fully connected feed-forward network. We employ a residual connection [10] around each of the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension dmodel = 512. Decoder: Thedecoder is also composed of a stack of N = 6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization. We also modify the self-attention sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This masking, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position i can depend only on the known outputs at positions less than i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key. 3.2.1 Scaled Dot-Product Attention We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the 3 Scaled Dot-Product Attention Multi-Head Attention Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel. query with all keys, divide each by dk, and apply a softmax function to obtain the weights on the values. In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q. The keys and values are also packed together into matrices K and V . We compute the matrix of outputs as: Attention(QKV ) = softmax(QKT dk )V (1) The two most commonly used attention functions are additive attention [2], and dot-product (multi plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor of 1 dk . Additive attention computes the compatibility function using a feed-forward network with a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is much faster and more space-efficient in practice, since it can be implemented using highly optimized matrix multiplication code. While for small values of dk the two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for larger values of dk [3]. We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients 4. To counteract this effect, we scale the dot products by 1 3.2.2 Multi-Head Attention dk . Instead of performing a single attention function with dmodel-dimensional keys, values and queries, we found it beneficial to linearly project the queries, keys and values h times with different, learned linear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of queries, keys and values we then perform the attention function in parallel, yielding dv-dimensional output values. These are concatenated and once again projected, resulting in the final values, as depicted in Figure 2. Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this. 4To illustrate why the dot products get large, assume that the components of q and k are independent random variables with mean 0 and variance 1. Then their dot product, q k = dk i=1 qiki, has mean 0 and variance dk. 4 MultiHead(QKV) = Concat(head1 headh)WO where headi = Attention(QWQ i KWK i VWV i ) Where the projections are parameter matrices WQ i and WO Rhdv dmodel. Rdmodel dk, WK i Rdmodel dk, WV i Rdmodel dv In this work we employ h = 8 parallel attention layers, or heads. For each of these we use dk = dv =dmodel h = 64. Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality. 3.2.3 Applications of Attention in our Model The Transformer uses multi-head attention in three different ways: In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [31, 2, 8]. The encoder contains self-attention layers. In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder. Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position. We need to prevent leftward information flow in the decoder to preserve the auto-regressive property. We implement this inside of scaled dot-product attention by masking out (setting to ) all values in the input of the softmax which correspond to illegal connections. See Figure 2. 3.3 Position-wise Feed-Forward Networks In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between. FFN(x) = max(0xW1 +b1)W2 +b2 (2) While the linear transformations are the same across different positions, they use different parameters from layer to layer. Another way of describing this is as two convolutions with kernel size 1. The dimensionality of input and output is dmodel = 512, and the inner-layer has dimensionality dff = 2048. 3.4 Embeddings and Softmax Similarly to other sequence transduction models, we use learned embeddings to convert the input tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transfor mation and softmax function to convert the decoder output to predicted next-token probabilities. In our model, we share the same weight matrix between the two embedding layers and the pre-softmax linear transformation, similar to [24]. In the embedding layers, we multiply those weights by dmodel. 3.5 Positional Encoding Since our model contains no recurrence and no convolution, in order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the 5 Table 1: Maximumpath lengths, per-layer complexity and minimum number of sequential operations for different layer types. n is the sequence length, d is the representation dimension, k is the kernel size of convolutions and r the size of the neighborhood in restricted self-attention.as the embeddings, so that the two can be summed. There are many choices of positional encodings, learned and fixed [8]. In this work, we use sine and cosine functions of different frequencies: PE(pos2i) = sin(pos 100002i dmodel) PE(pos2i+1) = cos(pos 100002i dmodel) where pos is the position and i is the dimension. That is, each dimension of the positional encoding corresponds to a sinusoid. The wavelengths form a geometric progression from 2 to 10000 2 . We chose this function because we hypothesized it would allow the model to easily learn to attend by relative positions, since for any fixed offset k, PEpos+k can be represented as a linear function of PEpos. Wealso experimented with using learned positional embeddings [8] instead, and found that the two versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version because it may allow the model to extrapolate to sequence lengths longer than the ones encountered during training. 4 WhySelf-Attention In this section we compare various aspects of self-attention layers to the recurrent and convolu tional layers commonly used for mapping one variable-length sequence of symbol representations (x1 xn) to another sequence of equal length (z1 zn), with xi zi Rd, such as a hidden layer in a typical sequence transduction encoder or decoder. Motivating our use of self-attention we consider three desiderata. One is the total computational complexity per layer. Another is the amount of computation that can be parallelized, as measured by the minimum number of sequential operations required. The third is the path length between long-range dependencies in the network. Learning long-range dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the ability to learn such dependencies is the length of the paths forward and backward signals have to traverse in the network. The shorter these paths between any combination of positions in the input and output sequences, the easier it is to learn long-range dependencies [11]. Hence we also compare the maximum path length between any two input and output positions in networks composed of the different layer types. Asnoted in Table 1, a self-attention layer connects all positions with a constant number of sequentially executed operations, whereas a recurrent layer requires O(n) sequential operations. In terms of computational complexity, self-attention layers are faster than recurrent layers when the sequence length n is smaller than the representation dimensionality d, which is most often the case with sentence representations used by state-of-the-art models in machine translations, such as word-piece [31] and byte-pair [25] representations. To improve computational performance for tasks involving very long sequences, self-attention could be restricted to considering only a neighborhood of size r in 6 the input sequence centered around the respective output position. This would increase the maximum path length to O(nr). We plan to investigate this approach further in future work. Asingle convolutional layer with kernel width k < n does not connect all pairs of input and output positions. Doing so requires a stack of O(nk) convolutional layers in the case of contiguous kernels, or O(logk(n)) in the case of dilated convolutions [15], increasing the length of the longest paths between any two positions in the network. Convolutional layers are generally more expensive than recurrent layers, by a factor of k. Separable convolutions [6], however, decrease the complexity considerably, to O(k n d + n d2). Even with k = n, however, the complexity of a separable convolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer, the approach we take in our model. Asside benefit, self-attention could yield more interpretable models. We inspect attention distributions from our models and present and discuss examples in the appendix. Not only do individual attention heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic and semantic structure of the sentences. 5 Training This section describes the training regime for our models. 5.1 Training Data and Batching We trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million sentence pairs. Sentences were encoded using byte-pair encoding [3], which has a shared source target vocabulary of about 37000 tokens. For English-French, we used the significantly larger WMT 2014 English-French dataset consisting of 36M sentences and split tokens into a 32000 word-piece vocabulary [31]. Sentence pairs were batched together by approximate sequence length. Each training batch contained a set of sentence pairs containing approximately 25000 source tokens and 25000 target tokens. 5.2 Hardware and Schedule We trained our models on one machine with 8 NVIDIA P100 GPUs. For our base models using the hyperparameters described throughout the paper, each training step took about 0.4 seconds. We trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the bottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps (3.5 days). 5.3 Optimizer Weused the Adam optimizer [17] with 1 = 09, 2 = 098 and = 10 9. Wevaried the learning rate over the course of training, according to the formula: lrate = d 05 model min(step_num 05 step_numwarmup_steps 15) (3) This corresponds to increasing the learning rate linearly for the first warmup_steps training steps, and decreasing it thereafter proportionally to the inverse square root of the step number. We used warmup_steps = 4000. 5.4 Regularization Weemploy three types of regularization during training: Residual Dropout We apply dropout [27] to the output of each sub-layer, before it is added to the sub-layer input and normalized. In addition, we apply dropout to the sums of the embeddings and the positional encodings in both the encoder and decoder stacks. For the base model, we use a rate of Pdrop = 01. 7 Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the English-to-German and English-to-French newstest2014 tests at a fraction of the training cost.hurts perplexity, as the model learns to be more unsure, but improves accuracy and BLEU score. 6 Results 6.1 Machine Translation On the WMT2014English-to-German translation task, the big transformer model (Transformer (big) in Table 2) outperforms the best previously reported models (including ensembles) by more than 20 BLEU, establishing a new state-of-the-art BLEU score of 284. The configuration of this model is listed in the bottom line of Table 3. Training took 35 days on 8 P100 GPUs. Even our base model surpasses all previously published models and ensembles, at a fraction of the training cost of any of the competitive models. On the WMT2014English-to-French translation task, our big model achieves a BLEU score of 410, outperforming all of the previously published single models, at less than 1 4 the training cost of the previous state-of-the-art model. The Transformer (big) model trained for English-to-French used dropout rate Pdrop = 01, instead of 03. For the base models, we used a single model obtained by averaging the last 5 checkpoints, which were written at 10-minute intervals. For the big models, we averaged the last 20 checkpoints. We used beam search with a beam size of 4 and length penalty = 06 [31]. These hyperparameters were chosen after experimentation on the development set. We set the maximum output length during inference to input length + 50, but terminate early when possible [31]. Table 2 summarizes our results and compares our translation quality and training costs to other model architectures from the literature. We estimate the number of floating point operations used to train a model by multiplying the training time, the number of GPUs used, and an estimate of the sustained single-precision floating-point capacity of each GPU 5. 6.2 Model Variations To evaluate the importance of different components of the Transformer, we varied our base model in different ways, measuring the change in performance on English-to-German translation on the development set, newstest2013. We used beam search as described in the previous section, but no checkpoint averaging. We present these results in Table 3. In Table 3 rows (A), we vary the number of attention heads and the attention key and value dimensions, keeping the amount of computation constant, as described in Section 3.2.2. While single-head attention is 0.9 BLEU worse than the best setting, quality also drops off with too many heads. 5We used values of 2.8, 3.7, 6.0 and 9.5 TFLOPS for K80, K40, M40 and P100, respectively. 8 Table3:VariationsontheTransformerarchitecture.Unlistedvaluesareidenticaltothoseofthebase model.AllmetricsareontheEnglish-to-Germantranslationdevelopmentset,newstest2013.Listed perplexitiesareper-wordpiece,accordingtoourbyte-pairencoding,andshouldnotbecomparedto per-wordperplexities
7 Conclusion Inthiswork,wepresentedtheTransformer,thefirstsequencetransductionmodelbasedentirelyon attention,replacingtherecurrentlayersmostcommonlyusedinencoder-decoderarchitectureswith multi-headedself-attention. Fortranslationtasks, theTransformercanbetrainedsignificantlyfasterthanarchitecturesbased onrecurrentorconvolutional layers. OnbothWMT2014English-to-GermanandWMT2014 English-to-Frenchtranslationtasks,weachieveanewstateoftheart. Intheformertaskourbest modeloutperformsevenallpreviouslyreportedensembles. Weareexcitedaboutthefutureofattention-basedmodelsandplantoapplythemtoothertasks.We plantoextendtheTransformertoproblemsinvolvinginputandoutputmodalitiesotherthantextand toinvestigatelocal,restrictedattentionmechanismstoefficientlyhandlelargeinputsandoutputs suchasimages,audioandvideo.Makinggenerationlesssequentialisanotherresearchgoalsofours. The codewe used to train and evaluate ourmodels is available at https://github.com/ tensorflow/tensor2tensor. Acknowledgements Wearegrateful toNalKalchbrennerandStephanGouwsfortheirfruitful comments,correctionsandinspiration.'''

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
tokenizer = Tokenizer()

In [4]:
tokenizer.fit_on_texts([Data])

In [5]:
tokenizer.word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'in': 5,
 'attention': 6,
 'a': 7,
 'we': 8,
 'is': 9,
 'model': 10,
 'for': 11,
 '2': 12,
 'models': 13,
 'sequence': 14,
 'layer': 15,
 'with': 16,
 'on': 17,
 '3': 18,
 'as': 19,
 'this': 20,
 'layers': 21,
 'our': 22,
 'output': 23,
 'encoder': 24,
 'decoder': 25,
 'positions': 26,
 '1': 27,
 'input': 28,
 'each': 29,
 'self': 30,
 'by': 31,
 'training': 32,
 'values': 33,
 'are': 34,
 'transformer': 35,
 'two': 36,
 '5': 37,
 'dot': 38,
 'all': 39,
 'used': 40,
 'dk': 41,
 'position': 42,
 'at': 43,
 'head': 44,
 'function': 45,
 'different': 46,
 'length': 47,
 'that': 48,
 'an': 49,
 'sub': 50,
 'recurrent': 51,
 '4': 52,
 'from': 53,
 'product': 54,
 'multi': 55,
 'also': 56,
 'translation': 57,
 'english': 58,
 'single': 59,
 '8': 60,
 'n': 61,
 'i': 62,
 'k': 63,
 'table': 64,
 'or': 65,
 'these': 66,
 'bleu': 67,
 'between': 68,
 'use': 69,
 'number': 70,
 'it': 71,
 'dmodel': 72,
 'than': 73,
 'keys': 74,
 'network': 75,
 'be': 76,

In [6]:
len(tokenizer.word_index)

1018

In [7]:
input_sequences = []
for sentence in Data.split('.'):
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [8]:
input_sequences

[[452, 453],
 [452, 453, 14],
 [452, 453, 14, 87],
 [452, 453, 14, 87, 13],
 [452, 453, 14, 87, 13, 34],
 [452, 453, 14, 87, 13, 34, 204],
 [452, 453, 14, 87, 13, 34, 204, 17],
 [452, 453, 14, 87, 13, 34, 204, 17, 454],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65, 126],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65, 126, 88],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65, 126, 88, 89],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65, 126, 88, 89, 48],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65, 126, 88, 89, 48, 455],
 [452, 453, 14, 87, 13, 34, 204, 17, 454, 51, 65, 126, 88, 89, 48, 455, 49],
 [452,
  453,
  14,
  87,
  13,
  34,
  204,
  17,
  454,
  51,
  65,
  126,
  88,
  89,
  48,
  455,
  49,
  24],
 [452,
  453,
  14,
  87,
  13,
  34,
  204,
  17,
  454,
  51,
  65,
  126,
  88,
  89,
  48,
  455,
  49,
  24,
  3],
 [452,
  453,
  14,
  87,
  13,
  34,
  204,
  17

In [9]:
max_len = max([len(x) for x in input_sequences])

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [11]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,  452,  453],
       [   0,    0,    0, ...,  452,  453,   14],
       [   0,    0,    0, ...,  453,   14,   87],
       ...,
       [   0,    0,    0, ..., 1014, 1015, 1016],
       [   0,    0,    0, ..., 1015, 1016, 1017],
       [   0,    0,    0, ..., 1016, 1017, 1018]], dtype=int32)

In [12]:
X = padded_input_sequences[:,:-1]

In [13]:
X

array([[   0,    0,    0, ...,    0,    0,  452],
       [   0,    0,    0, ...,    0,  452,  453],
       [   0,    0,    0, ...,  452,  453,   14],
       ...,
       [   0,    0,    0, ...,    0, 1014, 1015],
       [   0,    0,    0, ..., 1014, 1015, 1016],
       [   0,    0,    0, ..., 1015, 1016, 1017]], dtype=int32)

In [14]:
y = padded_input_sequences[:,-1]

In [15]:
y

array([ 453,   14,   87, ..., 1016, 1017, 1018], dtype=int32)

In [16]:
X.shape

(3605, 58)

In [17]:
y.shape

(3605,)

In [18]:
temp = len(tokenizer.word_index)

In [19]:
temp

1018

In [20]:
len(y)

3605

In [21]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=temp+1)


In [22]:
y.shape

(3605, 1019)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [24]:

input_length = 58

model1 = Sequential()
model1.add(Embedding(input_dim=temp + 1, output_dim=150, input_length=input_length))
model1.add(LSTM(150, return_sequences=True))
model1.add(LSTM(150))
model1.add(Dense(temp + 1, activation='softmax'))




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
model1.fit(X,y,epochs=100)

Epoch 1/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0581 - loss: 6.4454
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0708 - loss: 5.9205
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0675 - loss: 5.8482
Epoch 4/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0687 - loss: 5.6828
Epoch 5/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0822 - loss: 5.6266
Epoch 6/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0883 - loss: 5.4689
Epoch 7/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0944 - loss: 5.3778
Epoch 8/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.1133 - loss: 5.2164
Epoch 9/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1213 - loss: 5.1237
Epoch 10/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.1406 - loss: 4.8930
Epoch 11/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1473 - loss: 4.8014
Epoch 12/100
113/113 ━━━━━━━━━━━━━━━━

In [30]:
import numpy as np

In [34]:
import time
text = "We propose a new simple network architecture "

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model1.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
We propose a new simple network architecture  the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
We propose a new simple network architecture  the transformer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
We propose a new simple network architecture  the transformer based
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
We propose a new simple network architecture  the transformer based solely
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
We propose a new simple network architecture  the transformer based solely on
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
We propose a new simple network architecture  the transformer based solely on attention
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
We propose a new simple network architecture  the transformer based solely on attention mechanisms
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
We propose a new simple network architecture  the transformer based solely on attention mechanisms dispensing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
We propose a new simple netw